<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/Phase_envelopes_of_oil_and_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Phase enevelopes of oil and gas
#@markdown Demonstration of phase envelopes of gas and oil. This notebook installs NeqSim and imports the nessesary packages, and then runs calculations in Python/Colab.
#@markdown <br><br>This document is part of the module ["Introduction to Gas Processing using NeqSim in Colab"](https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/examples_of_NeqSim_in_Colab.ipynb#scrollTo=_eRtkQnHpL70).
%%capture
!pip install neqsim
import neqsim
from neqsim import jneqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math

## Reservoir types
A multicomponent mixture exhibits an envelope for liquid/vapor phase change in the pressure/temperature diagram, which contains a bubble-point line and a dew-point line, compared with only a phase change line for a pure component. The various reservoir types of oil and gas systems based on the phase behavior of hydrocarbons in the reservoir, in which the following five types of reservoirs are distinguished:
* Black oils;
* Volatile oils;
* Condensate (retrograde gases);
* Wet gases;
* Dry gases.

The amount of heavier molecules in the hydrocarbon mixtures varies from high to low in the black oils to the dry gases, respectively.

In [ ]:
#@title Introduction to Phase Envelopes
#@markdown This video gives an intriduction to behavour phase envelopes and phase changes and why this is Important
from IPython.display import YouTubeVideo
YouTubeVideo('K05k9uGapFs', width=600, height=400)

#Calculation of phase behaviour of single components
A single component boils at constant temperature for a given pressure. This is represented by the boiling point line in a PT-diagram.

In the form below we calculate the vapour pressure curve of a single component from the triple point temperature up to the critical point. SRK of PR-EoS can be used as basis for the calculations.

In [ ]:
eosname = 'srk' #@param ["srk", "pr"]
camponentName = "CO2" #@param ["CO2", "nitrogen", "methane", "ethane", "propane", "i-butane", "n-butane","n-pentane","n-hexane","n-heptane","n-octane"]
fluid1 = fluid('eosname') #create a fluid using the SRK-EoS
fluid1.addComponent(camponentName, 1.0) #adding 1 mole methane to the fluid

TTrip = fluid1.getPhase(0).getComponent(camponentName).getTriplePointTemperature()
PTrip = fluid1.getPhase(0).getComponent(camponentName).getTriplePointPressure()
Tcritical = fluid1.getPhase(0).getComponent(camponentName).getTC()
Pcritical = fluid1.getPhase(0).getComponent(camponentName).getPC()

fluid1.setTemperature(TTrip)
fluid1.setPressure(PTrip)
print('triple point temperature ', TTrip, "[K] and pressure ", PTrip, "[bara]")
print('critical temperature ', Tcritical, "[K] and pressure ", Pcritical, "[bara]")

def bubleP(pressure):
    fluid1.setPressure(pressure)
    bubt(fluid1)
    return fluid1.getTemperature('C')

pressure = np.arange(PTrip, Pcritical-5.0, 1.0)
temperature = [bubleP(P) for P in pressure]

plt.plot(temperature, pressure);
plt.xlabel('Temperature [C]');
plt.ylabel('Pressure [bara]');

#Calculation of Tx and Px diagrams of binary mixtures
Calculation of VLE phase diagrams of binary mixtures. In a P-x diagram, the bubble point and dew point curves bound the two-phase region at its top and its bottom, respectively. The single-phase liquid region is found at high pressures; the single-phase vapor region is found at low pressures. In the T-x diagram, this happens in the reverse order; vapor is found at high temperatures and liquid at low temperatures. Consequently, the bubble point and dew point curve are found at the bottom and the top of the two-phase region, respectively.

In the form below you can calculate the PX and TX diagram of a binary fluid. Select two components, the pressure of the TX diagram and the temperature of  the PX diagram.

In [ ]:
bubtemp = []
dewtemp = []

bubpres = []
dewpres = []

x = []
y = []
x2 = []
y2 = []
comp1 = "i-butane" #@param ["methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane","n-hexane"]
comp2 = "propane" #@param ["methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane","n-hexane"]
TXpressure = 11.0  #@param {type:"number"}
PXtemperature = -20.0  #@param {type:"number"}

fluid1 = fluid('srk')
addComponent(fluid1, comp1,0.000001)
addComponent(fluid1, comp2,1.01)
fluid1.setMixingRule('classic')
fluid1.setTemperature(Tcritical/2.0)
fluid1.setPressure(TXpressure)

fluid2 = fluid('srk')
addComponent(fluid2, comp1,0.000001)
addComponent(fluid2, comp2,1.01)
fluid2.setMixingRule('classic')
fluid2.setTemperature(PXtemperature+273.15)
fluid2.setPressure(Pcritical/20.0)

try:
    for temp in range(0,11):
        bubtemp.append(bubt(fluid1)-273.15)
        dewtemp.append(dewt(fluid1)-273.15)
        fluid1.init(0)
        x.append(fluid1.getPhase(0).getComponent(0).getz())
        fluid1.addComponent(comp2, -0.1)
        fluid1.addComponent(comp1, 0.1)
except:
        print('no dewt found')
try:
    for temp in range(0,11):
        bubpres.append(bubp(fluid2))
        dewpres.append(dewp(fluid2))
        fluid2.init(0)
        fluid2.addComponent(comp2, -0.1)
        fluid2.addComponent(comp1, 0.1)
except:
        print('no dewt found')

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(x, bubtemp, label='bubT')
plt.plot(x, dewtemp, label="dewT")
plt.xlabel('x [mol fraction comp 1]')
plt.ylabel('Temperature [C]')

plt.subplot(2, 1, 2)
plt.plot(x, bubpres, label='bubP')
plt.plot(x, dewpres, label="dewP")
plt.xlabel('x [mol fraction comp 1]')
plt.ylabel('Pressure [bara]')

#Calculation of phase envelope of a lean gas reservoir
Phase Envelope (two-phase region): The region enclosed by the bubble-point curve and the dew point curve wherein gas and liquid coexist in equlibrium, is identified as the phase envelope of the hydrocarbon system. A gas reservoir is dominated by light components, and generally contain low amounts of C7+ components.

In the form below you can specify the molar composition of a lean natural gas (heaviest component is n-hexane). The composition will be normalizes to make sure sum of molefractions is 1.

In [ ]:
nitrogen = 1.0 #@param {type:"number"}
CO2 = 2.5 #@param {type:"number"}
methane = 80.0  #@param {type:"number"}
ethane = 5.0  #@param {type:"number"}
propane =  2.5 #@param {type:"number"}
ibutane =  1.25 #@param {type:"number"}
nbutane =  1.25 #@param {type:"number"}
ipentane =  0.5 #@param {type:"number"}
npentane =  0.5 #@param {type:"number"}
nhexane =  0.05 #@param {type:"number"}

fluid1 = fluid('srk')
fluid1.addComponent("nitrogen", nitrogen)
fluid1.addComponent("CO2", CO2)
fluid1.addComponent("methane", methane)
fluid1.addComponent("ethane", ethane)
fluid1.addComponent("propane", propane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-butane", nbutane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-pentane", nbutane)
fluid1.addComponent("n-pentane", nbutane)
fluid1.addComponent("n-hexane", nbutane)
fluid1.setMixingRule('classic');

thermoOps = jneqsim.thermodynamicoperations.ThermodynamicOperations(fluid1)
thermoOps.calcPTphaseEnvelope2()

plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

#Calculation of phase envelope of a gas-condensate reservoir
A gas-condesate reservoir has more   heavy components, and generally contain hogh amounts of C7+ components.

In the form below you can specify the molar composition of a gas-condesate reservoir fluid (heaviest component is C7+). The composition will be normalizes to make sure sum of molefractions is 1.

In [ ]:
nitrogen = 0.51 #@param {type:"number"}
CO2 = 2.51 #@param {type:"number"}
methane = 80.0  #@param {type:"number"}
ethane = 4.01  #@param {type:"number"}
propane =  3.01 #@param {type:"number"}
ibutane =  1.25 #@param {type:"number"}
nbutane =  1.25 #@param {type:"number"}
ipentane =  0.25 #@param {type:"number"}
npentane = 0.25 #@param {type:"number"}
nhexane =  0.125 #@param {type:"number"}
C7 =  .305 #@param {type:"number"}
C7Molarmass =  150.0 #@param {type:"number"}
C7Density =  810.2 #@param {type:"number"}

fluid1 = fluid('pr')
fluid1.addComponent("nitrogen", nitrogen)
fluid1.addComponent("CO2", CO2)
fluid1.addComponent("methane", methane)
fluid1.addComponent("ethane", ethane)
fluid1.addComponent("propane", propane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-butane", nbutane)
fluid1.addComponent("i-pentane", ipentane)
fluid1.addComponent("n-pentane", npentane)
fluid1.addComponent("n-hexane", nhexane)
fluid1.addPlusFraction("C7", C7, C7Molarmass/1000.0, C7Density/1000.0)
fluid1.getCharacterization().characterisePlusFraction()
fluid1.setMixingRule('classic')
thermoOps = jneqsim.thermodynamicoperations.ThermodynamicOperations(fluid1)
thermoOps.calcPTphaseEnvelope2()

plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

# Calculation of phase envelope of an oil reservoir

In [ ]:
nitrogen = 0.51 #@param {type:"number"}
CO2 = 2.51 #@param {type:"number"}
methane = 20.0  #@param {type:"number"}
ethane = 4.01  #@param {type:"number"}
propane =  3.01 #@param {type:"number"}
ibutane =  1.25 #@param {type:"number"}
nbutane =  1.25 #@param {type:"number"}
ipentane =  0.25 #@param {type:"number"}
npentane = 0.25 #@param {type:"number"}
nhexane =  0.125 #@param {type:"number"}
C7 =  24.05 #@param {type:"number"}
C7Molarmass =  210.0 #@param {type:"number"}
C7Density =  810.2 #@param {type:"number"}

fluid1 = fluid('pr')
fluid1.addComponent("nitrogen", nitrogen)
fluid1.addComponent("CO2", CO2)
fluid1.addComponent("methane", methane)
fluid1.addComponent("ethane", ethane)
fluid1.addComponent("propane", propane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-butane", nbutane)
fluid1.addComponent("i-pentane", ipentane)
fluid1.addComponent("n-pentane", npentane)
fluid1.addComponent("n-hexane", nhexane)
fluid1.addPlusFraction("C7", C7, C7Molarmass/1000.0, C7Density/1000.0)
fluid1.getCharacterization().characterisePlusFraction()
fluid1.setMixingRule('classic')
thermoOps = jneqsim.thermodynamicoperations.ThermodynamicOperations(fluid1)
thermoOps.calcPTphaseEnvelope2()

plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

# Calculation of the phase envelope of an export gas
In the following example we calculate the phase envelope of a typical export gas. The cricondentherm and cricondebar will be reported.

In [ ]:
import pandas as pd

naturalgas = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane","i-pentane", "n-pentane","2-m-C5", "3-m-C5", "n-hexane", "benzene", "c-hexane", "n-heptane", "toluene", "c-C7", "n-octane", "m-Xylene", "c-C8", "n-nonane", "nC10"],
       'MolarComposition[-]':  [1.1799,0.5102,95.3303,2.1102,0.3217,0.1278,0.0846,0.0694,0.0340,0.0335,0.0109,0.0181,0.0017,0.0661,0.0207,0.0045,0.0530,0.0061,0.0033,0.000103,0.0032,0.0004]
}


naturalgasdf = pd.DataFrame(naturalgas)
print("Natural Gas Fluid:\n")
print(naturalgasdf.head(30).to_string())

naturalgasFluid = fluid_df(naturalgasdf)

gasPhaseEnvelope = jneqsim.thermodynamicoperations.ThermodynamicOperations(fluid1)
gasPhaseEnvelope.calcPTphaseEnvelope2()
cricobar = gasPhaseEnvelope.get("cricondenbar")
cricotherm = gasPhaseEnvelope.get("cricondentherm")
print("cricobarP ",  cricobar[1], "  [bara] ", " cricobarT ", cricobar[0], " K")
print("cricothermP ",  cricotherm[1], "  [bara] ", " cricothermT ", cricotherm[0], " K")
plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

In [ ]:

naturalgas = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane","i-pentane", "n-pentane"],
       'MolarComposition[-]':  [1.1799,0.5102,95.3303,2.1102,0.3217, 0.1 , 0.1, 0.1, 0.1]
}


naturalgasdf = pd.DataFrame(naturalgas)
print("Natural Gas Fluid:\n")
print(naturalgasdf.head(30).to_string())

naturalgasFluid = fluid_df(naturalgasdf)

gasPhaseEnvelope = phaseenvelope(naturalgasFluid,True)
cricobar = gasPhaseEnvelope.get("cricondenbar")
cricotherm = gasPhaseEnvelope.get("cricondentherm")
print("cricobarP ",  cricobar[1], "  [bara] ", " cricobarT ", cricobar[0], " °C")
print("cricothermP ",  cricotherm[1], "  [bara] ", " cricothermT ", cricotherm[0], " °C")

print('critical temperature  [K] ', gasPhaseEnvelope.get("criticalPoint1")[0], ' critical pressure  [bara] ', gasPhaseEnvelope.get("criticalPoint1")[1] )

#Using the UMR-PRU EoS
The UMR-PRU EoS is used in the PhaseOpt tool to calculate cricondenbar and dew points of rich natural gas.

http://members.igu.org/old/IGU%20Events/igrc/igrc-2014/papers/wo4-4_skouras.pdf

In [ ]:
fluid0_HC = fluid('umr')
fluid0_HC.addComponent('nitrogen',1.50384212276727)
fluid0_HC.addComponent('CO2',0.529539570730345)
fluid0_HC.addComponent('methane',92.8303477134498)
fluid0_HC.addComponent('ethane',3.4970377157638)
fluid0_HC.addComponent('propane',0.418221197906017)
fluid0_HC.addComponent('i-butane',0.380054178331476)
fluid0_HC.addComponent('n-butane',0.0582006527706574)
fluid0_HC.addComponent('22-dim-C3',0.00515867585725002)
fluid0_HC.addComponent('i-pentane',0.0620607111935889)
fluid0_HC.addComponent('n-pentane',0.0158347521527336)
fluid0_HC.addComponent('c-C5',0.0257579825598445)
fluid0_HC.addComponent('22-dim-C4',0.00931692731809784)
fluid0_HC.addComponent('23-dim-C4',0.0180174411828985)
fluid0_HC.addComponent('2-m-C5',0.0437804423449465)
fluid0_HC.addComponent('3-m-C5',0.0252426204923337)
fluid0_HC.addComponent('n-hexane',0.00697254561926368)
fluid0_HC.addComponent('benzene',0.000666939146190439)
fluid0_HC.addComponent('c-hexane',0.246403134725436)
fluid0_HC.addComponent('n-heptane',0.0422848348039577)
fluid0_HC.addComponent('toluene',0.00176839925126253)
fluid0_HC.addComponent('c-C7',0.19305790959197)
fluid0_HC.addComponent('n-octane',0.0123787686303101)
fluid0_HC.addComponent('m-Xylene',0.013156796366694)
fluid0_HC.addComponent('c-C8',0.00339071314488895)
fluid0_HC.addComponent('n-nonane',0.00758916287507585)
fluid0_HC.addComponent('nC10',0.000193663073477182)
fluid0_HC.addComponent('nC11',1.0e-6)
fluid0_HC.addComponent('nC12',1.0e-8)
fluid0_HC.setMixingRule("HV", "UNIFAC_UMRPRU")

thermoOps = jneqsim.thermodynamicoperations.ThermodynamicOperations(fluid0_HC)
thermoOps.calcPTphaseEnvelope2()

plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()



#sPhaseOpt for dew point calculations

In [ ]:
#...

#Mapping the phase types of a fluid
In the followinfg example we map the types of phase in a PT diagram. We also use it to check stability and speed of TPflash calculations.

In [ ]:
from neqsim import jneqsim
from neqsim.thermo import TPflash, fluid
import numpy as np
import time

testSystem = fluid('srk');
testSystem.addComponent("water", 65.93229747922976);
testSystem.addComponent("CO2", 0.784426208131475);
testSystem.addComponent("nitrogen", 1.578509157534656);
testSystem.addComponent("methane", 42.584113183429718);
testSystem.addComponent("ethane", 12.43870686718215);
testSystem.addComponent("propane", 1.26487350163523365);
testSystem.addComponent("i-butane", 1.04039429848533373);
testSystem.addComponent("n-butane", 1.1543856425679738);
testSystem.addComponent("i-pentane", 0.04039429848533373);
testSystem.addComponent("n-pentane", 0.1543856425679738);
testSystem.addTBPfraction("C6", 0.568724470114871, 84.93298402237961 / 1000.0,
    666.591171644071 / 1000.0);
testSystem.addTBPfraction("C7", 0.9478147516962493, 90.01311937418495 / 1000.0,
    746.9101810251765 / 1000.0);
testSystem.addTBPfraction("C8", 0.974840433764089, 102.34691375809437 / 1000.0,
    776.2927119017166 / 1000.0);
testSystem.addTBPfraction("C9", 0.5505907716430188, 116.06055719132209 / 1000.0,
    791.2983315058531 / 1000.0);
testSystem.addTBPfraction("C10", 1.9704404325720026, 221.831957 / 1000.0, 842.802708 / 1000.0);
testSystem.setMixingRule("classic");
testSystem.setMultiPhaseCheck(True);

data = np.zeros([100,100],dtype=np.uint8)
pressures = []
temperatures = []
start = time.time()

for i in range(100):
  testSystem.setPressure(i*5+1.0, "bara")
  pressures.append(testSystem.getPressure('bara'))
  temperatures = []
  for j in range(100):
    testSystem.setTemperature(j*5, "C")
    temperatures.append(testSystem.getTemperature('C'))
    try:
      TPflash(testSystem)
      data[i][j] = testSystem.getNumberOfPhases()
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 4
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 5
      if(not testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 6
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 7
      if(testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 8
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 9
    except:
       data[i][j] = 10

end  = time.time()
time = end-start

print('time per flash ', time/(100*100)*1000, ' msec')

import matplotlib.pyplot as plt
import numpy as np

# Assuming the data array and other variables are already populated

# Define a dictionary to map data values to region names
region_labels = {
    4: 'Aqueous-Oil',
    5: 'Oil only',
    6: 'Gas only',
    7: 'Gas-Oil-Aqueous',
    8: 'Gas-Aqueous',
    9: 'Gas-Oil'
}

# Dictionary to store the coordinates of all points in each region
region_points = {key: [] for key in region_labels.keys()}

# Collect all points for each region
for i in range(len(temperatures)):
    for j in range(len(pressures)):
        region_value = data[i][j]
        if region_value in region_points:
            region_points[region_value].append((temperatures[j], pressures[i]))

# Plot the data
plt.pcolormesh(temperatures, pressures, data, cmap='viridis')
plt.xlabel('Temperature [C]')
plt.ylabel('Pressure [bara]')

# Add one label at the center of each region
for region_value, points in region_points.items():
    if points:
        # Calculate the centroid of the region
        avg_temp = np.mean([p[0] for p in points])
        avg_pres = np.mean([p[1] for p in points])
        # Place the label at the centroid
        plt.text(avg_temp, avg_pres, region_labels[region_value], fontsize=10, ha='center', va='center', color='white')

plt.show()


We will do yet another example with less water in the feed.

In [ ]:
from neqsim import jneqsim
from neqsim.thermo import TPflash
import numpy as np
import time

testSystem = jneqsim.thermo.system.SystemSrkEos(273.15 + 100.0, 200.0);
testSystem.addComponent("water", 0.93229747922976);
#testSystem.addComponent("NaCl", 0.784426208131475);
testSystem.addComponent("nitrogen", 0.578509157534656);
testSystem.addComponent("CO2", 0.62803519623519);
testSystem.addComponent("methane", 22.5841131834297);
testSystem.addComponent("ethane", 3.43870686718);
testSystem.addComponent("propane", 0.26487350163523);
testSystem.addComponent("i-butane", 0.0403942984853);
testSystem.addComponent("n-butane", 0.1543856425679);
testSystem.addComponent("i-pentane", 0.243047568676360);
testSystem.addComponent("n-pentane", 0.338800037101);
testSystem.addTBPfraction("C6", 0.568724470114871, 84.93298402237961 / 1000.0,
    666.591171644071 / 1000.0);
testSystem.addTBPfraction("C7", 0.9478147516962493, 90.01311937418495 / 1000.0,
    746.9101810251765 / 1000.0);
testSystem.addTBPfraction("C8", 0.974840433764089, 102.34691375809437 / 1000.0,
    776.2927119017166 / 1000.0);
testSystem.addTBPfraction("C9", 0.5505907716430188, 116.06055719132209 / 1000.0,
    791.2983315058531 / 1000.0);
testSystem.addTBPfraction("C10", 12.970440432572002, 221.831957 / 1000.0, 842.802708 / 1000.0);
testSystem.setMixingRule("classic");
testSystem.setMultiPhaseCheck(True);

data = np.zeros([100,100],dtype=np.uint8)
pressures = []


start = time.time()

for i in range(data.shape[0]):
  pres = i*5+1.0
  testSystem.setPressure(pres, "bara")
  pressures.append(pres)
  temperatures = []

  for j in range(data.shape[1]):
    temp = j*5
    testSystem.setTemperature(temp, "C")
    temperatures.append(testSystem.getTemperature('C'))
    try:
      TPflash(testSystem)
      testSystem.initProperties()
      data[i][j] = testSystem.getNumberOfPhases()
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 4
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 5
      if(not testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 6
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 7
      if(testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 8
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 9
    except:
       data[i][j] = 10

end  = time.time()
time = end-start

print('time per flash ', time/(100*100)*1000, ' msec')


import matplotlib.pyplot as plt
import numpy as np

# Assuming the data array and other variables are already populated

# Define a dictionary to map data values to region names
region_labels = {
    4: 'Aqueous-Oil',
    5: 'Oil only',
    6: 'Gas only',
    7: 'Gas-Oil-Aqueous',
    8: 'Gas-Aqueous',
    9: 'Gas-Oil'
}

# Dictionary to store the coordinates of all points in each region
region_points = {key: [] for key in region_labels.keys()}

# Collect all points for each region
for i in range(100):
    for j in range(100):
        region_value = data[i][j]
        if region_value in region_points:
            region_points[region_value].append((temperatures[j], pressures[i]))

# Plot the data
plt.pcolormesh(temperatures, pressures, data, cmap='viridis')
plt.xlabel('Temperature [C]')
plt.ylabel('Pressure [bara]')

# Add one label at the center of each region
for region_value, points in region_points.items():
    if points:
        # Calculate the centroid of the region
        avg_temp = np.mean([p[0] for p in points])
        avg_pres = np.mean([p[1] for p in points])
        # Place the label at the centroid
        plt.text(avg_temp, avg_pres, region_labels[region_value], fontsize=10, ha='center', va='center', color='white')

plt.show()


# Phase behaviour of methane - n-heptane
Setting methane-n-heptane kij and running TPflash to evaluate number of phases

In [ ]:
from neqsim import jneqsim
from neqsim.thermo import TPflash
import time
import numpy as np
kij = 0.05

x_y_points = 200

testSystem = jneqsim.thermo.system.SystemPrEos(); # other models can be selected from: https://github.com/equinor/neqsim/tree/master/src/main/java/neqsim/thermo/system
testSystem.addComponent("methane", 70.0);
testSystem.addComponent("n-heptane", 30.0);

testSystem.setMixingRule("classic");

testSystem.getPhase(0).getMixingRule().setBinaryInteractionParameter(0, 1, kij)
testSystem.getPhase(1).getMixingRule().setBinaryInteractionParameter(0, 1, kij)

testSystem.setMultiPhaseCheck(True);

data = np.zeros([x_y_points,x_y_points],dtype=np.uint8)
pressures = []


start = time.time()

for i in range(data.shape[0]):
  pres = i*2.5
  testSystem.setPressure(pres, "bara")
  pressures.append(pres)
  temperatures = []

  for j in range(data.shape[1]):
    temp = 100 + j*2.0
    testSystem.setTemperature(temp, "K")
    temperatures.append(testSystem.getTemperature('K'))
    try:
      TPflash(testSystem)
      testSystem.initProperties()
      data[i][j] = testSystem.getNumberOfPhases()
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 4
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and not testSystem.hasPhaseType('gas')):
        data[i][j] = 5
      if(not testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 6
      if(testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 7
      if(testSystem.hasPhaseType('aqueous') and not testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 8
      if(not testSystem.hasPhaseType('aqueous') and testSystem.hasPhaseType('oil') and testSystem.hasPhaseType('gas')):
        data[i][j] = 9
      if testSystem.hasPhaseType('oil') and testSystem.getNumberOfPhases() == 2 and not testSystem.hasPhaseType('gas') and not testSystem.hasPhaseType('aqueous'):
            data[i][j] = 10
    except:
       data[i][j] = 11

end  = time.time()
time = end-start

print('time per flash ', time/(x_y_points*x_y_points)*1000, ' msec')


import matplotlib.pyplot as plt
import numpy as np

# Assuming the data array and other variables are already populated

# Define a dictionary to map data values to region names
region_labels = {
    4: 'Aqueous-Oil',
    5: 'Oil only',
    6: 'Gas only',
    7: 'Gas-Oil-Aqueous',
    8: 'Gas-Aqueous',
    9: 'Gas-Oil',
    10: 'Oil-Oil'
}

# Dictionary to store the coordinates of all points in each region
region_points = {key: [] for key in region_labels.keys()}

# Collect all points for each region
for i in range(x_y_points):
    for j in range(x_y_points):
        region_value = data[i][j]
        if region_value in region_points:
            region_points[region_value].append((temperatures[j], pressures[i]))

# Plot the data
plt.pcolormesh(temperatures, pressures, data, cmap='viridis')
plt.xlabel('Temperature [K]')
plt.ylabel('Pressure [bara]')

# Add one label at the center of each region
for region_value, points in region_points.items():
    if points:
        # Calculate the centroid of the region
        avg_temp = np.mean([p[0] for p in points])
        avg_pres = np.mean([p[1] for p in points])
        # Place the label at the centroid
        plt.text(avg_temp, avg_pres, region_labels[region_value], fontsize=10, ha='center', va='center', color='white')

plt.show()